In [1]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
import json
from prompts.common import SYSTEM_PROMPT
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from utils.common import to_markdown

In [2]:
TABLE_GENERATION_PROMPT = """
Based on the table metadata and schema generate a sql query to answer the user question
MetaData:
TableName: name
Description: Contains the name for each restaurant in the  area of the user
Columns:
id: contains unique id for each restaurant
name: contains name of restaurant

TableName:
TableName: details
Description: Contains all details regarding the restaurants menu
Columns:
restaurant_id: Foreign key that links this table with table `name`
item_type: Categories the item [Starters, Main Course, Desserts]
item_name: Name of item
description: Description regarding the item
ingredients: Ingredients of each item. Always lowercase this column values while querying.
cost: Cost of each item in $
preparation_time: Time taken to prepare each item in minutes

Schema:
create table restaurants.name(
    id int,
    name VARCHAR(255)
);

create table restaurants.details(
restaurant_id int,
item_type VARCHAR(50),
item_name VARCHAR(255),
description TEXT,
ingredients TEXT,
cost double,
preparation_time int
);

<<Example 1>>
User Query: What are the restaurants in my area

Response:
SELECT * FROM restaurants.name

<<Example 2>>
User Query: What is the menu for the restaurant Bistro Fusion?
    
Response:
SELECT * FROM restaurants.details
WHERE restaurant_id in (SELECT id FROM restaurants.name WHERE lower(name) like "%bistro fusion%")

<<Example 3>>
User Query: {input}

Response:
"""

In [3]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0613")

In [4]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are skilled SQL Engineer who can extract relevant entities from a question and convert that into a SQL query"),
    ("user", TABLE_GENERATION_PROMPT)
])

In [5]:
output_parser = StrOutputParser()

In [6]:
user_query = "Could you tell me dishes that get ready within 10 minutes"
# question = .format(input=user_query)

In [7]:
from utils.sql import MySQL

In [8]:
mysql = MySQL()

In [9]:
def process_query(query):
    data = mysql(query)
    return json.dumps(data)

In [10]:
sql_chain = {"input": RunnablePassthrough()}  | prompt | llm | output_parser | process_query

In [11]:
output = sql_chain.invoke(user_query)
print(output)

[{"restaurant_id": "0", "item_type": "Starters", "item_name": "Golden Corn Soup", "description": "A creamy, comforting blend of sweet corn and herbs.", "ingredients": "Sweet corn, Vegetable stock, Cream, Herbs", "cost": 5.99, "preparation_time": 10}, {"restaurant_id": "0", "item_type": "Starters", "item_name": "Beef Carpaccio", "description": "Thinly sliced raw beef dressed with olive oil, lemon juice, and capers.", "ingredients": "Beef, Olive oil, Lemon juice, Capers", "cost": 9.99, "preparation_time": 10}, {"restaurant_id": "1", "item_type": "Starters", "item_name": "Shrimp Cocktail", "description": "Chilled shrimp served with a zesty cocktail sauce.", "ingredients": "Shrimp, Cocktail sauce, Lemon", "cost": 8.99, "preparation_time": 10}, {"restaurant_id": "1", "item_type": "Starters", "item_name": "Bruschetta al Pomodoro", "description": "Toasted bread topped with fresh tomatoes, basil, and garlic.", "ingredients": "Bread, Tomatoes, Basil, Garlic", "cost": 6.5, "preparation_time": 10

In [12]:
query_result = process_query(output)
query_result

Error executing query: (1064, 'You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near \'[{"restaurant_id": "0", "item_type": "Starters", "item_name": "Golden Corn Soup"\' at line 1')


'null'

In [13]:
llm2 = ChatOpenAI(model="gpt-4-turbo")

In [19]:
template = ChatPromptTemplate.from_messages(
    [
        SYSTEM_PROMPT,
            ("human",
            """ 
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
If the context is empty then tell the user that no such data exists
Mention the dish type
Please format your response in Markdown. Include any necessary bullet points, tables, highlights, bold and italic text to enhance clarity and emphasis where needed.
Render tables without code
Question: {question}
Context: {context}
Answer:""".strip(),
        ),
    ]
)

main_chain = ( 
    template
    | llm2
    | StrOutputParser()
)

chain = (
    RunnableParallel({
        "context": sql_chain,
        "question": RunnablePassthrough(),
    }).assign(answer=main_chain)
)

In [20]:
out = chain.invoke(user_query)

In [21]:
out

{'context': '[{"restaurant_id": "0", "item_type": "Starters", "item_name": "Golden Corn Soup", "description": "A creamy, comforting blend of sweet corn and herbs.", "ingredients": "Sweet corn, Vegetable stock, Cream, Herbs", "cost": 5.99, "preparation_time": 10}, {"restaurant_id": "0", "item_type": "Starters", "item_name": "Beef Carpaccio", "description": "Thinly sliced raw beef dressed with olive oil, lemon juice, and capers.", "ingredients": "Beef, Olive oil, Lemon juice, Capers", "cost": 9.99, "preparation_time": 10}, {"restaurant_id": "1", "item_type": "Starters", "item_name": "Shrimp Cocktail", "description": "Chilled shrimp served with a zesty cocktail sauce.", "ingredients": "Shrimp, Cocktail sauce, Lemon", "cost": 8.99, "preparation_time": 10}, {"restaurant_id": "1", "item_type": "Starters", "item_name": "Bruschetta al Pomodoro", "description": "Toasted bread topped with fresh tomatoes, basil, and garlic.", "ingredients": "Bread, Tomatoes, Basil, Garlic", "cost": 6.5, "preparat

In [22]:
to_markdown(out["answer"])

> Here are the dishes that can be prepared within **10 minutes**:
> 
> ### Starters
> - **Golden Corn Soup**
>   - **Description**: A creamy, comforting blend of sweet corn and herbs.
>   - **Ingredients**: Sweet corn, Vegetable stock, Cream, Herbs
>   - **Cost**: $5.99
> 
> - **Beef Carpaccio**
>   - **Description**: Thinly sliced raw beef dressed with olive oil, lemon juice, and capers.
>   - **Ingredients**: Beef, Olive oil, Lemon juice, Capers
>   - **Cost**: $9.99
> 
> - **Shrimp Cocktail**
>   - **Description**: Chilled shrimp served with a zesty cocktail sauce.
>   - **Ingredients**: Shrimp, Cocktail sauce, Lemon
>   - **Cost**: $8.99
> 
> - **Bruschetta al Pomodoro**
>   - **Description**: Toasted bread topped with fresh tomatoes, basil, and garlic.
>   - **Ingredients**: Bread, Tomatoes, Basil, Garlic
>   - **Cost**: $6.5
> 
> ### Desserts
> - **Banana Split**
>   - **Description**: Classic banana split with scoops of vanilla, chocolate, and strawberry ice cream, topped with sauces, whipped cream, and a cherry.
>   - **Ingredients**: Banana, Ice cream, Sauces, Whipped cream, Cherry
>   - **Cost**: $5.99
> 
> These dishes are quick to prepare, making them perfect choices for a swift dining experience.

In [10]:
from chains.restaurant_data import RestaurantInfoChain
from langchain_openai import ChatOpenAI
from utils.common import to_markdown

In [2]:
rest_info_chain = RestaurantInfoChain()

In [6]:
user_query = "Could you suggest dishes that get ready within 10 minutes"

In [7]:
llm2 = ChatOpenAI(model="gpt-4-turbo")

In [11]:
result = rest_info_chain.process_prompt(llm2, user_query)

In [12]:
to_markdown(result["answer"])

> Here are some dishes from various restaurants that can be prepared within 10 minutes:
> 
> ### Starters
> | Item Name             | Description                                               | Ingredients                         | Cost  | Preparation Time |
> |-----------------------|-----------------------------------------------------------|-------------------------------------|-------|------------------|
> | **Golden Corn Soup**  | A creamy, comforting blend of sweet corn and herbs.       | Sweet corn, Vegetable stock, Cream, Herbs | $5.99 | 10 minutes       |
> | **Beef Carpaccio**    | Thinly sliced raw beef dressed with olive oil, lemon juice, and capers. | Beef, Olive oil, Lemon juice, Capers | $9.99 | 10 minutes       |
> | **Shrimp Cocktail**   | Chilled shrimp served with a zesty cocktail sauce.        | Shrimp, Cocktail sauce, Lemon       | $8.99 | 10 minutes       |
> | **Bruschetta al Pomodoro** | Toasted bread topped with fresh tomatoes, basil, and garlic. | Bread, Tomatoes, Basil, Garlic     | $6.50 | 10 minutes       |
> 
> ### Desserts
> | Item Name         | Description                                                                                         | Ingredients                                   | Cost  | Preparation Time |
> |-------------------|-----------------------------------------------------------------------------------------------------|-----------------------------------------------|-------|------------------|
> | **Banana Split**  | Classic banana split with scoops of vanilla, chocolate, and strawberry ice cream, topped with sauces, whipped cream, and a cherry. | Banana, Ice cream, Sauces, Whipped cream, Cherry | $5.99 | 10 minutes       |
> | **Mango Lassi**   | A sweet and refreshing yogurt-based mango drink.                                                    | Yogurt, Mango, Sugar, Cardamom                | $3.99 | 5 minutes        |
> 
> These dishes are quick to prepare and cover a range of flavors and preferences, from savory starters to sweet desserts.